## Linear Optimization - Supplier Selection Case

### Solution 1
Using "the list's length" as the first argument in the desicion variable, so the variable X will be 0, 1, 2, ...7.

In [1]:
import gurobipy as gb
from gurobipy import *
import numpy as np

In [2]:
#Data
mines_list = ['Ashley','Bedford','Consol','Dunby','Earlam','Florence','Gaston','Hopt']
mines_capacity = [300, 600, 510, 655, 575, 680, 450, 490] # (mt/yr)
mines_price = [49500, 50000, 61000, 63500, 66500, 71000, 72500, 80000] # ($/mt)

m = len(mines_list)
p = len(mines_price)
c = len(mines_capacity)

In [3]:
prob = gb.Model("Supplier Selection")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


In [4]:
X = prob.addVars(m, lb=0, ub=mines_capacity, vtype=GRB.CONTINUOUS, name = ["X_number of coal purchased from " +i for i in mines_list])

In [5]:
X

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>}

In [6]:
prob.setObjective(sum(mines_price[i]*X[i] for i in range(m)), GRB.MINIMIZE)

In [7]:
# A + B + C + D + E + F + G + H = 1225
prob.addConstr(sum(X[i] for i in range(m)) == 1225, name ="Procure of coking coal") 

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# A + B + D + F − C − E − G − H >= 0
union = [1, 1, -1, 1, -1, 1, -1, -1]

prob.addConstr(sum(X[i]*union[i] for i in range(m)) >= 0, name ="Coal from union mines")

<gurobi.Constr *Awaiting Model Update*>

In [9]:
# A + C + G + H <= 650
rail = [1, 0, 1, 0, 0, 0, 1, 1]

prob.addConstr(sum(X[i]*rail[i] for i in range(m)) <= 650, name ="transported via rail") 

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# B + D + E + F <= 720
trucks =[0, 1, 0, 1, 1, 1, 0, 0]

prob.addConstr(sum(X[i]*trucks[i] for i in range(m)) <= 720, name ="transported via trucks") 

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# −0.04A − 0.03B − 0.01C + 0.01D + 0.02E+ 0.03F + 0.04G + 0.06H >= 0
vol = [-0.04, -0.03, -0.01, 0.01, 0.02, 0.03, 0.04, 0.06]

prob.addConstr(sum(X[i]*vol[i] for i in range(m)) >= 0, name ="The average volatility")

<gurobi.Constr *Awaiting Model Update*>

In [12]:
for i in range(m):
    prob.addConstr(X[i] <= mines_capacity[i], name = f'capacity_{X[i]}')


In [13]:
prob.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 8 columns and 40 nonzeros
Model fingerprint: 0xfbaf32e1
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e+04, 8e+04]
  Bounds range     [3e+02, 7e+02]
  RHS range        [3e+02, 1e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0637500e+07   2.855000e+02   0.000000e+00      0s
       5    7.3267500e+07   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.326750000e+07


In [14]:
for v in prob.getVars():    
    print(v.varName, "=", round(v.x))
print(f"Optimal Objective Value: {prob.ObjVal}")

X_number of coal purchased from Ashley = 55
X_number of coal purchased from Bedford = 600
X_number of coal purchased from Consol = 0
X_number of coal purchased from Dunby = 20
X_number of coal purchased from Earlam = 100
X_number of coal purchased from Florence = 0
X_number of coal purchased from Gaston = 450
X_number of coal purchased from Hopt = 0
Optimal Objective Value: 73267500.0


### Solution 2
Using "list" as the first argument in the desicion variable, so the variable X will be the value in the list, such as X[0], X[1], ...X[7].

In [15]:
# Create a new model
prob = gb.Model("Supplier Selection")

# Data
mines_list = ['Ashley','Bedford','Consol','Dunby','Earlam','Florence','Gaston','Hopt']
mines_capacity = [300, 600, 510, 655, 575, 680, 450, 490] # (mt/yr)
mines_price = [49500, 50000, 61000, 63500, 66500, 71000, 72500, 80000]
union = [1, 1, -1, 1, -1, 1, -1, -1]
rail = [1, 0, 1, 0, 0, 0, 1, 1]
trucks =[0, 1, 0, 1, 1, 1, 0, 0]
vol = [-0.04, -0.03, -0.01, 0.01, 0.02, 0.03, 0.04, 0.06]

# Create variables
X = prob.addVars(mines_list, lb=0, ub=mines_capacity, vtype=GRB.CONTINUOUS, name = ["X_number of coal purchased from " +i for i in mines_list])

# Set objective
prob.setObjective(sum(mines_price[i]*X[j] for i, j in enumerate(mines_list)), GRB.MINIMIZE)

# Add constraints
prob.addConstr(sum(X[i] for i in mines_list) == 1225, name ="Procure of coking coal") 
prob.addConstr(sum(union[i]* X[j] for i,j in enumerate(mines_list)) >= 0, name ="Coal from union mines")
prob.addConstr(sum(rail[i]*X[j] for i,j in enumerate(mines_list)) <= 650, name ="transported via rail") 
prob.addConstr(sum(trucks[i]*X[i] for i,j in enumerate(mines_list)) <= 720, name ="transported via trucks") 
prob.addConstr(sum(vol[i]*X[j] for i,j in enumerate(mines_list)) >= 0, name ="The average volatility")

# Capacity constraints
for mine, capacity in zip(mines_list, mines_capacity):
    prob.addConstr(X[mine] <= capacity, f"capacity_{mine}")

# Non-negativity constraints are by default for the CONTINUOUS type.

# Optimize the model
prob.optimize()

# Display the results
if m.status == GRB.OPTIMAL:
    print('\nOptimal solution found:')
    for v in m.getVars():
        print(f"{v.varName} = {v.x}")
    print(f"Optimal Objective Value: {m.ObjVal}")
else:
    print('No solution found')


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 8 columns and 40 nonzeros
Model fingerprint: 0x72a5a292
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e+04, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0637500e+07   2.855000e+02   0.000000e+00      0s
       5    7.3267500e+07   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.326750000e+07

Optimal solution found:
X[A] = 55.0
X[B] = 600.0
X[C] = 0.0
X[D] = 19.999999999999943
X[E] = 100.00000000000006
X[F] = 0.0
X[G] = 450.0
X[H] = 0.0
Optimal Objective Value